In [1]:
# Init libraries
import warnings
import mne
import numpy as np
from sklearn.exceptions import ConvergenceWarning

np.random.seed(23)
mne.set_log_level(verbose='warning') #to avoid info at terminal
warnings.filterwarnings(action = "ignore", category = DeprecationWarning )
warnings.filterwarnings(action = "ignore", category = FutureWarning )
warnings.filterwarnings(action = "ignore", category = ConvergenceWarning )

In [2]:
# Project defaults
# The root dir
root_dir = "./ds003626"

# Sampling rate
fs = 256

# Select the useful par of each trial. Time in seconds
t_start = 1.5
t_end = 3.5

In [3]:
# Load dataset
from aux.pre_process import get_subjects_data_and_label, get_subjects_data_label_group

condition = "Inner"
data, labels, groups = get_subjects_data_label_group(root_dir, condition, t_start = t_start, t_end = t_end, fs = fs)
len(data), len(labels), len(groups)

(10, 10, 10)

In [4]:
data_array=np.vstack(data)
label_array=np.hstack(labels)
group_array=np.hstack(groups)

In [19]:
data_array.shape, label_array.shape, group_array.shape
print(data_array[0])

[[ 6.56632558e-06  9.29798573e-06  2.67945790e-06 ...  1.61658549e-05
   1.04383488e-05  1.65976586e-05]
 [ 6.73392992e-06  9.67485813e-06  3.46813168e-06 ...  1.72691653e-05
   1.09798877e-05  1.68224439e-05]
 [ 6.44456485e-06  1.05683281e-05  4.08183062e-06 ...  1.62432293e-05
   1.10782575e-05  1.67439421e-05]
 ...
 [ 7.73143200e-06  1.17620983e-05  5.96924569e-06 ...  9.61573311e-07
  -5.92098200e-06 -5.22468805e-06]
 [ 1.22760126e-05  1.52803701e-05  1.24073505e-05 ... -2.36157544e-06
  -1.04331358e-05 -6.78143929e-06]
 [ 1.31543686e-05  1.57064520e-05  1.26855428e-05 ... -1.16832456e-06
  -9.74268987e-06 -7.23316007e-06]]


In [10]:
from scipy import integrate
# Define all the features
from scipy import stats
import antropy as ant

def mean(x):
    return np.mean(x, axis=-1)

def std(x):
    return np.std(x, axis=-1)

def ptp(x):
    return np.ptp(x, axis=-1)

def var(x):
    return np.var(x, axis=-1)

def minim(x):
    return np.min(x, axis=-1)

def maxim(x):
    return np.max(x, axis=-1)

def argminim(x):
    return np. argmin(x, axis=-1)

def argmaxim(x):
    return np.argmax(x,axis=-1)

def rms(x):
    return np.sqrt(np.mean(x**2, axis=-1))

def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x, axis=-1)), axis=-1)

def skewness(x):
    return stats.skew(x, axis=-1)

def kurtosis(x):
    return stats.kurtosis(x, axis=-1)

def f_minplusmax(x):
    return np.max(x, axis=-1) + np.min(x, axis=-1)

def f_maxminusmin(x):
    return np.max(x, axis=-1) - np.min(x, axis=-1)

def f_spec_entropy(x):
    return ant.spectral_entropy(x, fs, method="welch", normalize=True, axis=-1)

def f_integral(x):
    return integrate.simps(x, axis=-1)

def f_petrosian(x):
    return ant.petrosian_fd(x, axis=-1)

def f_katz(x):
    return ant.katz_fd(x, axis=-1)

def concatenate_features(x):
    # Uncomment the desired line to add the feature
    return np.concatenate((
        mean(x),
        std(x),
        ptp(x),
        var(x),
        minim(x),
        maxim(x),
        argminim(x),
        argmaxim(x),
        rms(x),
        abs_diff_signal(x),
        skewness(x),
        kurtosis(x),
    ), axis=-1)

In [25]:
features=[]
for d in data_array:
    features.append(concatenate_features(d))
features_array=np.array(features)
print(features_array[0]))

(2236, 1536)

In [9]:
from sklearn.svm import LinearSVC

random_state = 42
splits = [0.10]

In [10]:
def run_cross_validation(classifier, k_fold, x_tr, y_tr, group):
    results = model_selection.cross_val_score(classifier, x_tr, y_tr, cv=k_fold, scoring='accuracy', groups=group)
    return results.mean()

In [11]:
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics, model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = features_array
y = label_array
X = StandardScaler().fit_transform(X)

# Feature Selection - There are other parameters we could set for Feature Selection
print("Old shape: ", X.shape)

# Select one fs here
fs = LinearSVC(C=0.01, penalty="l2", dual=False).fit(X, y)
model = SelectFromModel(fs, prefit=True)
X = model.transform(X)

print("New shape: ", X.shape)

Old shape:  (2236, 1536)
New shape:  (2236, 681)


In [12]:
from sklearn.model_selection import StratifiedGroupKFold

inner_cv = StratifiedGroupKFold(n_splits=5)
outer_cv = StratifiedGroupKFold(n_splits=5)

In [13]:
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

# Run Nested cross-validation

classifiers = [
    ["Linear SVC", LinearSVC(), {'C': [0.00001, 0.0001, 0.0005, 1, 10, 100, 1000], 'dual': (True, False)}],
    ["SVC", SVC(), [{"kernel": ["rbf"], "gamma": [1e-3, 1e-4], "C": [0.00001, 0.0001, 0.0005, 1, 10, 100, 1000]}, {"kernel": ["linear"], "C": [0.00001, 0.0001, 0.0005, 1, 10, 100, 1000]}, ]], # The third parameter here is an array
]

for test_size in splits:
    print("\nSplit: Train:{}% Test:{}%".format(100 - (test_size * 100), test_size * 100))

    # Stratify guarantees that the same proportion of the classes will be available in train and test
    x_train, x_test, y_train, y_test, g_train, g_test = train_test_split(X, y, group_array, test_size=test_size, stratify=y)

    for cls in classifiers:
        print('{}: {} '.format("Classifier", cls[0]))
        clf = GridSearchCV(estimator=cls[1], param_grid=cls[2], cv=inner_cv, n_jobs=-1)
        clf.fit(x_train, y_train, groups=g_train)
        print(f"The best parameters found are: {clf.best_params_}")
        print(f"The mean CV score of the best model is: {clf.best_score_:.3f}")

        print("Grid scores on development set:\n")
        means = clf.cv_results_["mean_test_score"]
        stds = clf.cv_results_["std_test_score"]
        for mean, std, params in zip(means, stds, clf.cv_results_["params"]):
            print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

        print("\nDetailed classification report:\n")
        print("The model is trained on the full development set.")
        print("The scores are computed on the full evaluation set.\n")
        y_true, y_pred = y_test, clf.predict(x_test)
        print(classification_report(y_true, y_pred))
        print()


Split: Train:90.0% Test:10.0%
Classifier: Linear SVC 


/home/rollin/anaconda3/envs/thesis/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/rollin/anaconda3/envs/thesis/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/rollin/anaconda3/envs/thesis/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/rollin/anaconda3/envs/thesis/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/rollin/anaconda3/envs/thesis/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/rollin/anaconda3/envs/thesis/lib/p

The best parameters found are: {'C': 1000, 'dual': True}
The mean CV score of the best model is: 0.324
Grid scores on development set:

0.271 (+/-0.024) for {'C': 1e-05, 'dual': True}
0.271 (+/-0.024) for {'C': 1e-05, 'dual': False}
0.278 (+/-0.043) for {'C': 0.0001, 'dual': True}
0.278 (+/-0.043) for {'C': 0.0001, 'dual': False}
0.296 (+/-0.051) for {'C': 0.0005, 'dual': True}
0.296 (+/-0.051) for {'C': 0.0005, 'dual': False}
0.320 (+/-0.027) for {'C': 1, 'dual': True}
0.318 (+/-0.053) for {'C': 1, 'dual': False}
0.315 (+/-0.047) for {'C': 10, 'dual': True}
0.317 (+/-0.042) for {'C': 10, 'dual': False}
0.315 (+/-0.035) for {'C': 100, 'dual': True}
0.316 (+/-0.039) for {'C': 100, 'dual': False}
0.324 (+/-0.045) for {'C': 1000, 'dual': True}
0.317 (+/-0.050) for {'C': 1000, 'dual': False}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

         

In [14]:
# Run the regular tests
classifiers = [
    ["Linear SVC", LinearSVC(random_state=random_state, max_iter=10000, C=10)],
    ["SVC", SVC(random_state=random_state, max_iter=10000, C=10, kernel='linear')],
]

for test_size in splits:
    print("\nSplit: Train:{}% Test:{}%".format(100 - (test_size * 100), test_size * 100))
    print('{:<40} {:<20} {:<15}'.format("Classifier", "Accuracy", "Cross validation"))

    # Stratify guarantees that the same proportion of the classes will be available in train and test
    x_train, x_test, y_train, y_test, g_train, g_test = train_test_split(X, y, group_array, test_size=test_size, stratify=y)

    for cls in classifiers:
        cls[1].fit(x_train, y_train)
        y_pred = cls[1].predict(x_test)
        accuracy = metrics.accuracy_score(y_test, y_pred)
        cross_v = run_cross_validation(cls[1], outer_cv, x_train, y_train, g_train)
        print('{:<40} {:<20} {:<15}'.format(cls[0], accuracy, cross_v))


Split: Train:90.0% Test:10.0%
Classifier                               Accuracy             Cross validation
Linear SVC                               0.34375              0.29961930282731164
SVC                                      0.29910714285714285  0.3143670678048091
